In [ ]:
import torch as th
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

from models import DINCAE_pconvs
from preprocessing.dataset_normalization_v2 import MinMaxNormalization

In [ ]:
result_idx = 95
dataset_idx = 1

data_path = Path("../data/")
dataset_path = data_path / Path(f"minimal_datasets/dataset_{dataset_idx}.pt")
weights_path = data_path / Path(f"weights/weights_{result_idx}.pt")
# weights_path = data_path / Path(f"weights_to_sync/weights_{result_idx}.pt")
nans_masks_path = data_path / Path(f"nans_masks/nans_mask_{dataset_idx}.pt")
minmax_path = data_path / Path(f"minmax_vals/minmax_{dataset_idx}.pt")

for path in [dataset_path, weights_path, nans_masks_path, minmax_path]:
    if not path.exists():
        raise FileNotFoundError(f"Path {path} does not exist.")

dataset = th.load(dataset_path)
weights = th.load(weights_path)
nans_masks = th.load(nans_masks_path)
minmax = th.load(minmax_path)

In [ ]:
# Retieve dataset specs
n_channels = dataset["images"].shape[1]
nrows = 128
ncols = 128
n_images = 200
masked_channels = [0, 1]

In [ ]:
model = DINCAE_pconvs(params_path=Path("params.json"))

model.load_state_dict(weights)

In [ ]:
# Get random image
random_index = np.random.choice(n_images, 1, replace=False)
channel = np.random.choice(masked_channels, 1, replace=False)

# random_index = [14]
# channel = 0

print(f"Random index: {random_index}")
print(f"Random channel: {channel}")

image = dataset["images"][random_index]
mask = dataset["masks"][random_index]
nans_mask = nans_masks[random_index]

if model.__class__.__name__ == "DINCAE_pconvs":
    output, _ = model(image, mask.float())
else:
    output = model(th.where(~mask, image, -1))

raw_output = output.clone()

image = th.where(~nans_mask, th.nan, image)

output = th.where(~mask, output, image) # Where mask is 0, use the output, else use the original image
output = th.where(~nans_mask, th.nan, output)
np_output = output[0, channel, :, :].detach().squeeze(0).numpy()
np_image = image[0, channel, :, :].detach().squeeze(0).numpy()
raw_output_np = raw_output[0, channel, :, :].detach().squeeze(0).numpy()

mask = mask.detach().squeeze().numpy()
nans_mask = nans_mask.detach().squeeze().numpy()

# Determine global vmin and vmax to ensure same colormap scaling
vmin = np.nanmin([np_output, np_image])
vmax = np.nanmax([np_output, np_image])

fig, axs = plt.subplots(1, 3, figsize=(15, 5))

im0 = axs[0].imshow(np_output, vmin=vmin, vmax=vmax)
axs[0].set_title("Reconstructed image")


im1 = axs[1].imshow(np_image, vmin=vmin, vmax=vmax)
axs[1].set_title("Original image")

im2 = axs[2].imshow(raw_output_np, vmin=vmin, vmax=vmax)
axs[2].set_title("Raw output")

# Shared colorbar using one of the images
fig.colorbar(im1, ax=axs, orientation='vertical')

plt.show()

In [ ]:
norm_class = MinMaxNormalization()
denormalized_raw_output = norm_class.denormalize(raw_output, minmax).detach().numpy()
denormalized_image = norm_class.denormalize(image, minmax).detach().numpy()
denormalized_output = norm_class.denormalize(output, minmax).detach().numpy()


In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
channel = 0

vmin = np.nanmin([denormalized_output[0, channel, :, :], denormalized_image[0, channel, :, :]])
vmax = np.nanmax([denormalized_output[0, channel, :, :], denormalized_image[0, channel, :, :]])
print(f"Reconstructed image min: {vmin}, max: {vmax}")
im0 = axs[0].imshow(denormalized_output[0, channel, :, :], vmin=vmin, vmax=vmax)
axs[0].set_title("Reconstructed image")
im1 = axs[1].imshow(denormalized_image[0, channel, :, :], vmin=vmin, vmax=vmax)
axs[1].set_title("Original image")
im2 = axs[2].imshow(denormalized_raw_output[0, channel, :, :], vmin=vmin, vmax=vmax)
axs[2].set_title("Raw output")

fig.colorbar(im1, ax=axs, orientation='vertical')
plt.show()
